# YOLOv3 with custom objects
These are the steps I followed on Google's Colab to train a yolov3 model on custom objects. My project involved **QR code localization** but besides the data and model configuration, the process is the same to train yolov3 on any custom object.

## Setting up the environment

In [ ]:
# Remove some of the default files from Codalab
!rm -rf sample_data

In [ ]:
%%shell
# Update and upgrade packages
apt-get update
apt-get upgrade

# Some dependencies & extra
apt-get -y install cmake
apt-get install libopencv-dev
apt-get install vim

# Compilers
apt install g++-5
apt install gcc-5

# CUDA
wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
apt-get install cuda -y -qq
apt install cuda-8.0 -y

In [ ]:
# Check that tensorflow-gpu recognizes your GPU
import tensorflow as tf
print(tf.test.gpu_device_name())

# Check CUDA installation and drivers
!nvcc --version
!nvidia-smi

## Darknet and weights

In [ ]:
# Get the Darknet repo
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
# Modify Makefile to include GPU usage
%%shell
cd darknet
sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
sed -i 's/GPU=0/GPU=1/g' Makefile

In [ ]:
# You can run this to make sure the Makefile was properly modified (stop manually)
%%shell
cd darknet
vim Makefile

Now that everything is set up, we need to compile the project. After compiling check that darknet.exe is in the main branch or in build/darknet/x64. If you can't find darknet.exe then the project didn't compile correctly

In [ ]:
%%shell
cd darknet
make

# If darknet.exe is in the main file move to x64
mv darknet build/darknet/x64

In [ ]:
# Download some default weights to the x64 folder
# YOLOv3 and YOLOv3 tiny weights
%cd darknet/build/darknet/x64
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://pjreddie.com/media/files/yolov3-tiny.weights
!./darknet partial yolov3-tiny-obj.cfg yolov3-tiny.weights  yolov3-tiny.conv.15 15

## Uploading files

The following files need to be added to certain directories of the _darknet/build/darknet/x64_ path. A short explanation of what they are is given but AlexeyAB's repository explains everything in more detail:
- **Weights** to _backup_ folder. These are not necessary and new ones will be generated for training, but you might want to start training on pretrained data or train a model for longer.
- **obj.data, obj.names, and train.txt** to _data_ folder. The first file contains general information about the dataset, the second one the label, and the last one the path to all images.
- **Images/Data** to _data/obj_. You may need to create or upload the _obj_ folder, this is where all your images will be located.
- **.cfg file** to _x64_. This is the configuration of the model, you can download it from the repo and modify it to fit your data.

In [ ]:
# Anchors for YOLOv3 and YOLOv3-tiny model
# These need to be modified for your desired resolution
# And for the data you will be using, then change anchors in .cfg file
!./darknet detector calc_anchors data/obj.data -num_of_clusters 9 -width 512 -height 512
!./darknet detector calc_anchors data/obj.data -num_of_clusters 6 -width 480 -height 480

## Training and testing
While training and testing you need to run the following commands while on _darknet/build/darknet/x64_, where your darknet.exe file is located. While training, the weights will be saved in the _backup_ folder every hundred iterations. The predictions will be saved as an image _predictions.jpg_. You will need to replace some of the parts in the command to include your desired weights and config file.

In [ ]:
# Training
!./darknet detector train data/obj.data yolov3-tiny-obj.cfg backup/yolov3-tiny-obj_last.weights -dont_show

In [ ]:
# Testing
!./darknet detector test data/obj.data yolov3-tiny-obj-test.cfg backup/yolov3-tiny-obj_4000.weights -ext_output